In [16]:
import zarr
import numpy as np
from os.path import join

In [17]:
from spatialdata import read_zarr

In [18]:
data_dir = "data"
sdata_path = join(data_dir, "xenium_rep1_io.spatialdata.zarr")
ngff_path = join(data_dir, "xenium_points.ome.zarr")

In [19]:
sdata = read_zarr(sdata_path)
ddf = sdata.points["transcripts"]
df = ddf.compute()

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/mkeller/research/dbmi/vitessce/point-cloud-exploration/.venv/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


In [20]:
z = zarr.open(ngff_path, mode="r")
img_arr = z['/0']

In [21]:
img_arr.chunks

(1, 471, 684)

In [43]:
chunk_shape = img_arr.chunks
first_chunk = img_arr[:, :chunk_shape[1], :chunk_shape[2]]

In [44]:
first_chunk

array([[[ 44.79042053,  45.90461731,  50.93250275, ..., 354.08953857,
         356.8527832 , 363.12747192],
        [378.03845215, 381.3364563 , 382.00964355, ..., 191.56343079,
         194.67765808, 206.65226746],
        [207.83117676, 209.07495117, 210.36459351, ..., 233.95748901,
         234.85700989, 237.41049194],
        ...,
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ]],

       [[595.84332275, 632.55426025, 296.1116333 , ..., 633.32312012,
         672.86102295, 597.88641357],
        [652.95556641, 441.11404419, 678.50195312, ..., 593.01159668,
         602.39837646, 667.2298584 ],
        [647.40289307, 630.24163818, 609.07568359, ..., 577.71411133,
         484.68688965, 672.6697998 ],
        ...,


In [45]:
scale_factor = 4.0 # TODO: get from metadata

In [46]:
z.attrs["multiscales"][0]["datasets"][0]["coordinateTransformations"]

[{'scale': [1.0, 4.0, 4.0], 'type': 'scale'},
 {'translation': [0.0, -1.8734135627746582, 4.415738582611084],
  'type': 'translation'}]

In [68]:
df["norm_x"] = (df["x"] - -1.8734135627746582) * scale_factor
df["norm_y"] = (df["y"] - 4.415738582611084) * scale_factor

In [69]:
first_chunk_df = df.loc[(df["norm_x"] <= chunk_shape[1]) & (df["norm_y"] <= chunk_shape[2])]

In [77]:
(44.790421 / scale_factor) + -1.8734135627746582

9.324191687225342

In [78]:
(595.843323 / scale_factor) + 4.415738582611084

153.3765693326111

In [70]:
first_chunk_df

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv,norm_x,norm_y
29,9.324192,153.376572,8.940705,ITGAX,490,0,281474976710685,40.000000,44.790421,595.843323
36,9.602740,162.554306,9.082570,PTRHD1,490,0,281474976710692,16.651773,45.904617,632.554260
40,10.859712,78.443649,7.236133,TCIM,480,0,281474976710696,40.000000,50.932503,296.111633
44,10.418414,167.996048,8.731602,GLIPR1,492,0,281474976710700,9.398639,49.167313,654.321228
55,11.866437,94.522766,10.064598,DAPK3,485,1,281474976710711,40.000000,54.959404,360.428101
...,...,...,...,...,...,...,...,...,...,...
881692,105.662468,130.266647,15.984596,ANGPT2,19013,0,281474977605576,8.257154,430.143524,503.403625
881694,105.593643,165.153793,15.190280,ANKRD30A,19010,0,281474977605578,40.000000,429.868225,642.952209
881755,108.429642,173.583237,12.367968,DST,19010,1,281474977605639,40.000000,441.212219,676.669983
881905,113.796791,167.498001,13.387850,NARS,19010,1,281474977605789,40.000000,462.680817,652.329041


In [71]:
first_chunk[0, :, :]

array([[ 44.79042053,  45.90461731,  50.93250275, ..., 354.08953857,
        356.8527832 , 363.12747192],
       [378.03845215, 381.3364563 , 382.00964355, ..., 191.56343079,
        194.67765808, 206.65226746],
       [207.83117676, 209.07495117, 210.36459351, ..., 233.95748901,
        234.85700989, 237.41049194],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]], shape=(471, 684))

In [75]:
float(first_chunk_df["norm_x"].sum())

1493970.5

In [76]:
float(first_chunk[0, :, :].sum())

1493970.4287548065

In [79]:
# Check the feature_index values.
first_chunk[2, :, :]

array([[153., 237., 284., ..., 170., 195., 170.],
       [311., 106., 170., ..., 126., 111., 111.],
       [256., 204.,   1., ..., 206., 183., 119.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], shape=(471, 684))